In [1]:
from typing import Iterable

import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import UnitaryGate
from scipy.linalg import expm

def random_unitary_iter(r1:int,r2:int)->Iterable:
    rham = np.random.rand(2**r2,2**r2) + 1j*np.random.rand(2**r2,2**r2)
    rham += np.conj(np.transpose(rham))
    irham = 1j*rham
    for m in range(r1):
        rut=expm((2**m)*irham)
        rut/np.trace(rut)
        yield rut

def controlled_U_gate(circ:QuantumCircuit,ctrl:int,targs:list,rut:np.ndarray)->None:
    rut_gate=UnitaryGate(rut)
    crut_gate = rut_gate.control(1)
    circ.append(crut_gate,qargs=[ctrl,*targs])

In [2]:
from typing import Tuple

from qiskit import QuantumRegister
from qiskit.result import Counts
from qiskit.circuit.library import QFT
from qiskit.quantum_info import Statevector
import matplotlib.pyplot as plt

def plot_counts(counts:Counts):
    plt.plot(
        range(len(counts.int_outcomes().values())),
        np.abs(list(counts.int_outcomes().values())),
        "rs:",
        markerfacecolor='none',
        label="counts"
    )
    plt.legend()
    plt.show()

def get_random_phase_and_psi(unitary:np.ndarray)->np.ndarray:
    eigvals,eigvecs=np.linalg.eig(unitary)
    idx = np.random.randint(0,len(eigvals))

    # eigval = e^{2pi i theta}
    eigval = eigvals[idx]
    phase = np.real(-1j * np.log(eigval)/(2*np.pi))
    phase = np.mod(phase,1)
    psi = eigvecs[:,idx]
    return phase,psi

def qpe_circuit(r1:int,r2:int)->Tuple[QuantumCircuit,float]:

    first_register=QuantumRegister(r1,"first")
    second_register=QuantumRegister(r2,"second")

    circ = QuantumCircuit(first_register,second_register)
    circ.h(first_register)

    unitaries = random_unitary_iter(r1,r2)

    get_phase_and_psi=True
    for ind,unitary in enumerate(unitaries):
        if get_phase_and_psi:
            phase,psi= get_random_phase_and_psi(unitary)
            get_phase_and_psi=False

        controlled_U_gate(circ=circ,ctrl=ind,targs=second_register,rut=unitary)

    circ.append(QFT(num_qubits=len(first_register),inverse=True,do_swaps=True),qargs=first_register)

    return circ,phase,psi

def bitstring_idx_to_phase(index)->float:
    bin(index)

def get_phase_from_qpe_circuit(circ:QuantumCircuit,psi:np.ndarray)->float:

    out_circ = circ.copy_empty_like()

    out_circ.prepare_state(state=psi,qubits=circ.qregs[1])
    out_circ.append(circ,qargs=circ.qubits)
    statevector = Statevector.from_instruction(out_circ)

    r1=len(out_circ.qregs[0])
    counts=statevector.sample_counts(1000,qargs=range(r1))
    bitstring =counts.most_frequent()

    return np.sum([(int(v))*2**-(i+1) for i,v in enumerate(bitstring)])

In [3]:
for trial in range(10):
    circ,phase,psi = qpe_circuit(1,2)
    computed_phase = get_phase_from_qpe_circuit(circ,psi)
    print(f"trial {trial} | correct: {phase:.5f}, guess: {computed_phase:.5f} err: {np.abs(phase-computed_phase):.5f}")

trial 0 | correct: 0.97163, guess: 0.00000 err: 0.97163
trial 1 | correct: 0.15066, guess: 0.00000 err: 0.15066
trial 2 | correct: 0.86997, guess: 0.00000 err: 0.86997
trial 3 | correct: 0.03735, guess: 0.00000 err: 0.03735
trial 4 | correct: 0.67710, guess: 0.50000 err: 0.17710
trial 5 | correct: 0.59421, guess: 0.50000 err: 0.09421
trial 6 | correct: 0.75082, guess: 0.00000 err: 0.75082
trial 7 | correct: 0.14367, guess: 0.00000 err: 0.14367
trial 8 | correct: 0.98424, guess: 0.00000 err: 0.98424
trial 9 | correct: 0.82495, guess: 0.00000 err: 0.82495
